# **SLT Mobitel Annual Report - SFT**

In [ ]:
!uv pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!uv pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!uv pip install transformers==4.51.3
!uv pip install --no-deps unsloth

Using Python 3.11.11 environment at: /usr
Resolved 8 packages in 91ms
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 16.00 KiB/22.14 KiB
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 22.14 KiB/22.14 KiB
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 22.14 KiB/22.14 KiB
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 22.14 KiB/22.14 KiB
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 22.14 KiB/22.14 KiB
unsloth-zoo ------------------------------ 32.00 KiB/142.36 KiB
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 22.14 KiB/22.14 KiB
unsloth-zoo ------------------------------ 32.00 KiB/142.36 KiB
trl        ------------------------------     0 B/311.46 KiB
⠙ Preparing packages... (0/5)
cut-cross-entropy ------------------------------ 22.14 Ki

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/Qwen2.5-0.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)



model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-02 06:45:17.399374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748846717.586160      76 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748846717.641409      76 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/521M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.9 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [ ]:
from datasets import Dataset

def load_my_text_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return {"text": [line.strip() for line in lines if line.strip()]}

dataset = Dataset.from_dict(load_my_text_dataset("/kaggle/input/greatmanith/textdata/textdata/annual.txt"))
print(dataset)

print(len(dataset))

Dataset({
    features: ['text'],
    num_rows: 24616
})
24616


In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",  # Key that holds your plain text
    max_seq_length = 1024,
    dataset_num_proc = 2,
    packing = False,              # Set True for many short sequences
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        num_train_epochs = 5,              # ✅ Use this instead of max_steps
        # max_steps = 100,                  # You can increase this for better results
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use "wandb" if you want logging
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/24616 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 24,616 | Num Epochs = 5 | Total steps = 3,845
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss
